In [1]:
import os
import sys
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="This sequence already has </s>.")

# Append path for module imports
scripts_path = os.path.abspath(os.path.join('..', 'scripts'))
sys.path.append(scripts_path)


# Standard library imports
import random
import string

# Third-party imports
import json
import numpy as np
import pandas as pd
import itertools
import pke
import torch
import nltk
from dateutil.parser import parse
from nltk.corpus import stopwords, wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from textdistance import levenshtein
from rapidfuzz.distance import Levenshtein as levenshtein

# Download necessary NLTK data
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')

from typing import List, Dict
import re


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Import custom models from model_manager
from model_manager import (
    get_answergeneration_model,
    get_questiongeneration_model,
    get_sense2vec_model,
    get_sentence_transformer_model,
    get_random_passage
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\01SetupFiles\AnacondaNavigator\anacondanavigatorfiles\envs\Thesis\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\01SetupFiles\AnacondaNavigator\anacondanavigatorfiles\envs\Thesis\Lib\site-packages\sentence_trans

In [3]:
# Initialize models
t5ag_model, t5ag_tokenizer = get_answergeneration_model()
t5qg_model, t5qg_tokenizer = get_questiongeneration_model()
s2v = get_sense2vec_model()
sentence_transformer_model = get_sentence_transformer_model()
random_passage = get_random_passage()

In [4]:
def answer_question(question, context):
    """Generate an answer for a given question and context."""
    input_text = f"question: {question} context: {context}"
    input_ids = t5ag_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        output = t5ag_model.generate(input_ids, max_length=512, num_return_sequences=1, max_new_tokens=200)

    return t5ag_tokenizer.decode(output[0], skip_special_tokens=True).capitalize()

def get_passage(passage):
    """Generate a random context from the dataset."""
    return passage.sample(n=1)['context'].values[0]

def get_question(context, answer, model, tokenizer):
    """Generate a question for the given answer and context."""
    answer_span = context.replace(answer, f"<hl>{answer}<hl>", 1) + "</s>"
    inputs = tokenizer(answer_span, return_tensors="pt")
    question = model.generate(input_ids=inputs.input_ids, max_length=50)[0]

    return tokenizer.decode(question, skip_special_tokens=True)

def get_nouns_multipartite(content):
    """Extract keywords from content using MultipartiteRank algorithm."""
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=content, language='en')
        pos_tags = {'PROPN', 'NOUN', 'ADJ', 'VERB', 'ADP', 'ADV', 'DET', 'CONJ', 'NUM', 'PRON', 'X'}

        stoplist = list(string.punctuation) + ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')

        extractor.candidate_selection(pos=pos_tags)
        extractor.candidate_weighting(alpha=1.1, threshold=0.75, method='average')
        keyphrases = extractor.get_n_best(n=15)
        
        return [val[0] for val in keyphrases]
    except Exception as e:
        print(f"Error extracting nouns: {e}")
        return []

def get_keywords(passage):
    """Extract keywords using TF-IDF."""
    try:
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform([passage])
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.toarray().flatten()
        word_scores = dict(zip(feature_names, tfidf_scores))
        sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
        keywords = [word for word, score in sorted_words]
        return keywords
    except Exception as e:
        print(f"Error extracting keywords: {e}")
        return []

def filter_same_sense_words(original, wordlist):
    """Filter words that have the same sense as the original word."""
    base_sense = original.split('|')[1]
    return [word[0].split('|')[0].replace("_", " ").title().strip() for word in wordlist if word[0].split('|')[1] == base_sense]

def get_max_similarity_score(wordlist, word):
    """Get the maximum similarity score between the word and a list of words."""
    return max(levenshtein.normalized_similarity(word.lower(), each.lower()) for each in wordlist)

def sense2vec_get_words(word, s2v, topn, question):
    """Get similar words using Sense2Vec."""
    try:
        sense = s2v.get_best_sense(word, senses=["NOUN", "PERSON", "PRODUCT", "LOC", "ORG", "EVENT", "NORP", "WORK_OF_ART", "FAC", "GPE", "NUM", "FACILITY"])
        
        if sense is None:
            print(f"[DEBUG] No suitable sense found for word: '{word}'")
            return []

        most_similar = s2v.most_similar(sense, n=topn)
        output = filter_same_sense_words(sense, most_similar)
        
        threshold = 0.6
        final = [word]
        checklist = question.split()

        for similar_word in output:
            if get_max_similarity_score(final, similar_word) < threshold and similar_word not in final and similar_word not in checklist:
                final.append(similar_word)
    
        return final[1:]
    
    except Exception as e:
        print(f"Error in Sense2Vec: {e}")
        return []

def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
    """Maximal Marginal Relevance (MMR) for keyword extraction."""
    try:
        word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
        word_similarity = cosine_similarity(word_embeddings)

        keywords_idx = [np.argmax(word_doc_similarity)]
        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

        for _ in range(top_n - 1):
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

            mmr = (lambda_param * candidate_similarities) - ((1 - lambda_param) * target_similarities.reshape(-1, 1))
            mmr_idx = candidates_idx[np.argmax(mmr)]

            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)

        return [words[idx] for idx in keywords_idx]
    except Exception as e:
        print(f"Error in MMR: {e}")
        return []
    
def classify_question_type(question: str) -> str:
    """
    Classify the type of question as literal, evaluative, or inferential.
    
    Parameters:
        question (str): The question to classify.
        
    Returns:
        str: The type of the question ('literal', 'evaluative', or 'inferential').
    """
    # Define keywords or patterns for each question type
    literal_keywords = [
    'what', 'when', 'where', 'who', 'how many', 'how much', 
    'which', 'name', 'list', 'identify', 'define', 'describe', 
    'state', 'mention'
    ]

    evaluative_keywords = [
    'evaluate', 'justify', 'explain why', 'assess', 'critique', 
    'discuss', 'judge', 'opinion', 'argue', 'agree or disagree', 
    'defend', 'support your answer', 'weigh the pros and cons', 
    'compare', 'contrast'
    ]

    inferential_keywords = [
    'why', 'how', 'what if', 'predict', 'suggest', 'imply', 
    'conclude', 'infer', 'reason', 'what might', 'what could', 
    'what would happen if', 'speculate', 'deduce', 'interpret', 
    'hypothesize', 'assume'
    ]


    question_lower = question.lower()
    
    # Check for literal question keywords
    if any(keyword in question_lower for keyword in literal_keywords):
        return 'literal'
    
    # Check for evaluative question keywords
    if any(keyword in question_lower for keyword in evaluative_keywords):
        return 'evaluative'
    
    # Check for inferential question keywords
    if any(keyword in question_lower for keyword in inferential_keywords):
        return 'inferential'
    
    # Default to 'unknown' if no pattern matches
    return 'unknown'

def get_distractors_wordnet(word):
    """Get distractors using WordNet."""
    distractors = []
    try:
        synsets = wn.synsets(word, pos='n')
        if not synsets:
            print(f"[DEBUG] No synsets found for word: '{word}'")
            return distractors

        synset = synsets[0]
        hypernyms = synset.hypernyms()
        if not hypernyms:
            print(f"[DEBUG] No hypernyms found for synset of word: '{word}'")
            return distractors

        hypernym = hypernyms[0]
        hyponyms = hypernym.hyponyms()
        if not hyponyms:
            print(f"[DEBUG] No hyponyms found for hypernym of word: '{word}'")
            return distractors

        for item in hyponyms:
            name = item.lemmas()[0].name().replace("_", " ").title()
            if name.lower() != word.lower() and name not in distractors:
                distractors.append(name)
    except Exception as e:
        print(f"Error in WordNet distractors: {e}")
    return distractors

def get_distractors(phrase, original_sentence, sense2vec_model, sentence_model, top_n, lambda_val):
    """Get distractors for a given phrase using various methods."""
    try:
        if any(char.isdigit() for char in phrase):
            # Handle as a date or numerical distractor
            if is_date(phrase):
                return generate_date_distractors(phrase)
            else:
                return generate_numerical_distractors(phrase)
        
        words = phrase.split()
        distractors = set()

        if len(words) == 1:
            word = words[0]
            distractors.update(get_distractors_for_single_word(word, original_sentence, sense2vec_model, top_n))
        else:
            word_distractors = []
            for word in words:
                word_distractors.append(list(get_distractors_for_single_word(word, original_sentence, sense2vec_model, top_n)))
            
            distractor_combinations = set()
            for combination in itertools.product(*word_distractors):
                combined_distractor = ' '.join(combination)
                if combined_distractor.lower() != phrase.lower() and len(combination) == len(words):
                    distractor_combinations.add(combined_distractor.capitalize())
            distractors.update(distractor_combinations)

        distractors.add(phrase.capitalize())
        embedding_sentence = f"{original_sentence} {phrase.capitalize()}"
        keyword_embedding = sentence_model.encode([embedding_sentence])
        distractor_embeddings = sentence_model.encode(list(distractors))
        max_keywords = min(len(distractors), 5)
        filtered_keywords = mmr(keyword_embedding, distractor_embeddings, list(distractors), max_keywords, lambda_val)
        final_distractors = [kw.capitalize() for kw in filtered_keywords if kw.lower() != phrase.lower()]
        return final_distractors
    except Exception as e:
        print(f"Error in getting distractors: {e}")
        return []

def get_distractors_for_single_word(word, original_sentence, sense2vec_model, top_n):
    """Helper function to get distractors for a single word."""
    distractors = set()
    try:
        distractors_word = sense2vec_get_words(word, sense2vec_model, top_n, original_sentence)
        if distractors_word:
            distractors.update(distractors_word)
        
        if not distractors:
            distractors_wordnet = get_distractors_wordnet(word)
            if distractors_wordnet:
                distractors.update(distractors_wordnet)
    except Exception as e:
        print(f"Error in single word distractors: {e}")
    return distractors

def is_date(string):
    """Check if a string is a date."""
    try:
        parse(string, fuzzy=False)
        return True
    except ValueError:
        return False

def generate_date_distractors(date_string):
    """Generate distractors for date phrases."""
    try:
        date = parse(date_string)
        distractors = set()
        for _ in range(5):
            year = date.year + random.choice([-5, -3, -1, 1, 3, 5])
            month = random.randint(1, 12)
            day = min(random.randint(1, 28), 28)
            new_date = date.replace(year=year, month=month, day=day)
            distractors.add(new_date.strftime("%m/%d/%Y"))
        return list(distractors)
    except Exception as e:
        print(f"Error generating date distractors: {e}")
        return []

def generate_numerical_distractors(number_string):
    """Generate distractors for numerical phrases."""
    try:
        # Parse the number from the string, handling integer and float types
        number = float(number_string) if '.' in number_string else int(number_string)
        distractors = set()

        # Generate random variations around the original number
        for _ in range(5):
            variation = random.uniform(-0.3, 0.3) * number
            new_number = number + variation
            distractors.add(round(new_number, 2))

        # Ensure the original number is not in the distractors
        distractors.discard(number)
        return list(distractors)
    except Exception as e:
        print(f"Error generating numerical distractors: {e}")
        return []



In [5]:
def get_mca_questions(context, qg_model, qg_tokenizer, s2v, sentence_transformer_model, num_questions=5, max_attempts=2) -> List[Dict]:
    """
    Generate multiple-choice questions for a given context.
    
    Parameters:
        context (str): The context from which questions are generated.
        qg_model (T5ForConditionalGeneration): The question generation model.
        qg_tokenizer (T5Tokenizer): The tokenizer for the question generation model.
        s2v (Sense2Vec): The Sense2Vec model for finding similar words.
        sentence_transformer_model (SentenceTransformer): The sentence transformer model for embeddings.
        num_questions (int): The number of questions to generate.
        max_attempts (int): The maximum number of attempts to generate questions.
    
    Returns:
        list: A list of dictionaries with questions and their corresponding distractors.
    """
    output_list = []

    imp_keywords = get_keywords(context)  # Extract keywords only once
    print(f"[DEBUG] Length: {len(imp_keywords)}, Extracted keywords: {imp_keywords}")

    generated_questions = set()
    generated_answers = set()  # Track generated answers
    attempts = 0

    while len(output_list) < num_questions and attempts < max_attempts:
        attempts += 1

        for keyword in imp_keywords:
            if len(output_list) >= num_questions:
                break
            
            question = get_question(context, keyword, qg_model, qg_tokenizer)
            print(f"[DEBUG] Generated question: '{question}' for keyword: '{keyword}'")
            
            # Encode the new question
            new_question_embedding = sentence_transformer_model.encode(question, convert_to_tensor=True)
            is_similar = False

            # Check similarity with existing questions
            for generated_q in generated_questions:
                existing_question_embedding = sentence_transformer_model.encode(generated_q, convert_to_tensor=True)
                similarity = cosine_similarity(new_question_embedding.unsqueeze(0), existing_question_embedding.unsqueeze(0))[0][0]

                if similarity > 0.8:
                    is_similar = True
                    print(f"[DEBUG] Question '{question}' is too similar to an existing question, skipping.")
                    break

            if is_similar:
                continue

            # Generate and check answer
            t5_answer = answer_question(question, context)
            print(f"[DEBUG] Generated answer: '{t5_answer}' for question: '{question}'")
            
            # Skip answers longer than 3 words
            if len(t5_answer.split()) > 3:
                print(f"[DEBUG] Answer '{t5_answer}' is too long, skipping.")
                continue

            if t5_answer in generated_answers:
                print(f"[DEBUG] Answer '{t5_answer}' has already been generated, skipping question.")
                continue

            generated_questions.add(question)
            generated_answers.add(t5_answer)

            # Generate distractors
            distractors = get_distractors(t5_answer, question, s2v, sentence_transformer_model, 40, 0.2)
            if len(distractors) == 0:
                print("[DEBUG] No distractors found, using important keywords as distractors.")
                distractors = imp_keywords

            # If answer is a phrase, handle distractors appropriately
            answer_words = t5_answer.split()
            distractors = [d for d in distractors if not any(word.lower() in t5_answer.lower() for word in answer_words)]
            distractors = [d.capitalize() for d in distractors if d.lower() != keyword.lower()]

            if len(distractors) < 3:
                additional_distractors = [kw.capitalize() for kw in imp_keywords if kw.capitalize() not in distractors and kw.lower() != keyword.lower()]
                distractors.extend(additional_distractors[:3 - len(distractors)])
            else:
                distractors = distractors[:3]

            print(f"[DEBUG] Final distractors: {distractors} for question: '{question}'")

            choices = distractors + [t5_answer]
            choices = [item.title() for item in choices]
            random.shuffle(choices)
            print(f"[DEBUG] Options: {choices} for question: '{question}'")

            # Classify question type
            question_type = classify_question_type(question)
            
            output_list.append({
                'answer': t5_answer,
                'answer_length': len(t5_answer),
                'choices': choices,
                'passage': context,
                'passage_length': len(context),
                'question': question,
                'question_length': len(question),
                'question_type': question_type
            })

        print(f"[DEBUG] Generated {len(output_list)} questions so far after {attempts} attempts")

    return output_list[:num_questions]

In [6]:
original_context = get_passage(random_passage)

print(original_context)

questions_and_distractors = get_mca_questions(original_context, t5qg_model, t5qg_tokenizer, s2v, sentence_transformer_model, num_questions=5)

OUTPUT_FILE = os.path.join('..', 'outputs', 'plots', 'predictions', 'generated_questions.json')

def convert_to_serializable(data):
    if isinstance(data, pd.Series):
        return data.tolist()
    elif isinstance(data, dict):
        return {k: convert_to_serializable(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_to_serializable(i) for i in data]
    else:
        return data

try:
    with open(OUTPUT_FILE, 'r') as f:
        existing_data = json.load(f)
        if not isinstance(existing_data, list):  # Ensure existing data is a list
            raise ValueError("The existing data is not a list.")
except FileNotFoundError:
    existing_data = []
except json.JSONDecodeError:
    print(f"Warning: {OUTPUT_FILE} contains invalid JSON. Initializing with an empty list.")
    existing_data = []
except ValueError as e:
    print(f"ValueError: {e}")
    existing_data = []
except Exception as e:
    print(f"An unexpected error occurred while reading {OUTPUT_FILE}: {e}")
    existing_data = []

# Convert any non-serializable data to a serializable format
questions_and_distractors_serializable = convert_to_serializable(questions_and_distractors)
existing_data.extend(questions_and_distractors_serializable)

# Write updated data back to the file
try:
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(existing_data, f, indent=4)
    print(f"Data successfully written to {OUTPUT_FILE}.")
except Exception as e:
    print(f"An error occurred while writing to {OUTPUT_FILE}: {e}")

On 12 January 1953, he was appointed Patriarch of Venice and, accordingly, raised to the rank of Cardinal-Priest of Santa Prisca by Pope Pius XII. Roncalli left France for Venice on 23 February 1953 stopping briefly in Milan and then to Rome. On 15 March 1953, he took possession of his new diocese in Venice. As a sign of his esteem, the President of France, Vincent Auriol, claimed the ancient privilege possessed by French monarchs and bestowed the red biretta on Roncalli at a ceremony in the Élysée Palace. It was around this time that he, with the aid of Monsignor Bruno Heim, formed his coat of arms with a lion of Saint Mark on a white ground. Auriol also awarded Roncalli three months later with the award of Commander of the Legion of Honour.
[DEBUG] Length: 68, Extracted keywords: ['1953', 'roncalli', 'venice', 'auriol', 'france', '12', '15', '23', 'accordingly', 'aid', 'ancient', 'appointed', 'arms', 'award', 'awarded', 'bestowed', 'biretta', 'briefly', 'bruno', 'cardinal', 'ceremony

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: '1953' for question: 'In what year was he appointed Patriarch of Venice?'
[DEBUG] Final distractors: ['Roncalli', 'Venice', 'Auriol'] for question: 'In what year was he appointed Patriarch of Venice?'
[DEBUG] Options: ['Auriol', 'Venice', 'Roncalli', '1953'] for question: 'In what year was he appointed Patriarch of Venice?'
[DEBUG] Generated question: 'What was the name of the diocese in which he was diocese?' for keyword: 'roncalli'


Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: 'Venice' for question: 'What was the name of the diocese in which he was diocese?'
[DEBUG] Final distractors: ['1953', 'Venice', 'Auriol'] for question: 'What was the name of the diocese in which he was diocese?'
[DEBUG] Options: ['Venice', 'Venice', 'Auriol', '1953'] for question: 'What was the name of the diocese in which he was diocese?'
[DEBUG] Generated question: 'What was the name of the diocese in which he was diocese?' for keyword: 'venice'
[DEBUG] Question 'What was the name of the diocese in which he was diocese?' is too similar to an existing question, skipping.
[DEBUG] Generated question: 'What was the name of the diocese in which he was diocese?' for keyword: 'auriol'
[DEBUG] Question 'What was the name of the diocese in which he was diocese?' is too similar to an existing question, skipping.
[DEBUG] Generated question: 'What was the name of the diocese in which he was diocese?' for keyword: 'france'
[DEBUG] Question 'What was the name of the dioc

...

[DEBUG] Generated answer: '1953' for question: 'When did Roncalli leave France for Venice?'
[DEBUG] Answer '1953' has already been generated, skipping question.
[DEBUG] Generated question: 'How did he become Patriarch of Venice?' for keyword: 'accordingly'
[DEBUG] Question 'How did he become Patriarch of Venice?' is too similar to an existing question, skipping.
[DEBUG] Generated question: 'What did Monsignor Bruno Heim provide to Roncalli?' for keyword: 'aid'


Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: 'Vincent auriol' for question: 'What did Monsignor Bruno Heim provide to Roncalli?'
[DEBUG] No suitable sense found for word: 'auriol'
[DEBUG] No synsets found for word: 'auriol'
[DEBUG] No distractors found, using important keywords as distractors.
[DEBUG] Final distractors: ['1953', 'Roncalli', 'Venice'] for question: 'What did Monsignor Bruno Heim provide to Roncalli?'
[DEBUG] Options: ['Vincent Auriol', '1953', 'Venice', 'Roncalli'] for question: 'What did Monsignor Bruno Heim provide to Roncalli?'
[DEBUG] Generated question: 'What privilege did Auriol claim that French monarchs had?' for keyword: 'ancient'


Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: 'Ancient privilege' for question: 'What privilege did Auriol claim that French monarchs had?'
[DEBUG] Final distractors: ['1953', 'Roncalli', 'Venice'] for question: 'What privilege did Auriol claim that French monarchs had?'
[DEBUG] Options: ['Venice', 'Ancient Privilege', 'Roncalli', '1953'] for question: 'What privilege did Auriol claim that French monarchs had?'
[DEBUG] Generated 5 questions so far after 1 attempts
Data successfully written to ..\outputs\plots\predictions\generated_questions.json.
